In [2]:
import pandas as pd

In [8]:
#data = pd.read_csv("..data/processed/cleaned_data.csv")
data = pd.read_csv("../data/processed/cleaned_data.csv")


C:\Users\smith\AppData\Local\Temp\ipykernel_2700\2337864690.py:2: DtypeWarning: Columns (31) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("../data/processed/cleaned_data.csv")


In [9]:
data

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,TransactionYear,TransactionQuarter,TransactionMonthNum
0,145249,12827,2015-03-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,2015,1,3
1,145249,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,2015,2,5
2,145249,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,2015,3,7
3,145255,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0,2015,2,5
4,145255,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0,2015,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000019,31520,389,2015-04-01,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,347.235175,0.0,2015,2,4
1000020,31520,389,2015-06-01,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,347.235175,0.0,2015,2,6
1000021,31520,389,2015-08-01,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,347.235175,0.0,2015,3,8
1000022,31519,389,2014-07-01,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,2.315000,0.0,2014,3,7


In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000024 entries, 0 to 1000023
Data columns (total 49 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   UnderwrittenCoverID       1000024 non-null  int64  
 1   PolicyID                  1000024 non-null  int64  
 2   TransactionMonth          1000024 non-null  object 
 3   IsVATRegistered           1000024 non-null  bool   
 4   Citizenship               1000024 non-null  object 
 5   LegalType                 1000024 non-null  object 
 6   Title                     1000024 non-null  object 
 7   Language                  1000024 non-null  object 
 8   Bank                      1000024 non-null  object 
 9   AccountType               1000024 non-null  object 
 10  MaritalStatus             1000024 non-null  object 
 11  Gender                    1000024 non-null  object 
 12  Country                   1000024 non-null  object 
 13  Province                  1

In [16]:
data.columns.values

array(['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth',
       'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language',
       'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country',
       'Province', 'PostalCode', 'MainCrestaZone', 'SubCrestaZone',
       'ItemType', 'mmcode', 'VehicleType', 'RegistrationYear', 'make',
       'Model', 'Cylinders', 'cubiccapacity', 'kilowatts', 'bodytype',
       'NumberOfDoors', 'VehicleIntroDate', 'AlarmImmobiliser',
       'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'SumInsured',
       'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected',
       'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product',
       'StatutoryClass', 'StatutoryRiskType', 'TotalPremium',
       'TotalClaims', 'TransactionYear', 'TransactionQuarter',
       'TransactionMonthNum'], dtype=object)

In [21]:
data['Group']

KeyError: 'Group'

In [19]:
import pandas as pd
from scipy.stats import ttest_ind, chi2_contingency

def calculate_claim_metrics(data):
    data = data.copy()
    data['HasClaim'] = data['TotalClaims'] > 0
    claim_freq = data.groupby('Group')['HasClaim'].mean()
    claim_severity = data[data['HasClaim']].groupby('Group')['TotalClaims'].mean()
    margin = data.groupby('Group').apply(lambda x: (x['TotalPremium'] - x['TotalClaims']).mean())
    return claim_freq, claim_severity, margin

def t_test_metric(data, metric):
    group_a = data[data['Group'] == 'A'][metric]
    group_b = data[data['Group'] == 'B'][metric]
    stat, p_value = ttest_ind(group_a, group_b, equal_var=False)
    return stat, p_value

def chi_squared_risk(data):
    contingency = pd.crosstab(data['Group'], data['HasClaim'])
    chi2, p, _, _ = chi2_contingency(contingency)
    return chi2, p

def interpret_result(p1, p2, p3):
    results = []
    if p1 < 0.05:
        results.append("→ Risk frequency difference is significant.")
    if p2 < 0.05:
        results.append("→ Claim severity difference is significant.")
    if p3 < 0.05:
        results.append("→ Margin difference is significant.")
    if not results:
        return "Fail to reject null hypothesis. No significant differences found."
    return " ".join(results)


def hypothesis_test_pipeline(data, group_col, hypothesis_name):
    data = data.copy()

    # Drop NA values in necessary columns
    data = data.dropna(subset=['TotalClaims', 'TotalPremium', group_col])

    # Select top 2 categories for grouping
    top_two = data[group_col].value_counts().index[:2]
    data = data[data[group_col].isin(top_two)]
    data['Group'] = data[group_col].apply(lambda x: 'A' if x == top_two[0] else 'B')

    # ✅ Add HasClaim here so it's available globally
    data['HasClaim'] = data['TotalClaims'] > 0

    print(f"\n--- Hypothesis: {hypothesis_name} ---")
    claim_freq, claim_severity, margin = calculate_claim_metrics(data)

    # Chi-squared on claim frequency
    chi2, p1 = chi_squared_risk(data)
    print(f"Chi-squared test (Claim Frequency): p = {p1:.4f}")

    # T-test on severity
    _, p2 = t_test_metric(data[data['HasClaim']], 'TotalClaims')
    print(f"T-test (Claim Severity): p = {p2:.4f}")

    # T-test on margin
    data['Margin'] = data['TotalPremium'] - data['TotalClaims']
    _, p3 = t_test_metric(data, 'Margin')
    print(f"T-test (Margin): p = {p3:.4f}")

    return {
        "hypothesis": hypothesis_name,
        "chi2_p": p1,
        "severity_p": p2,
        "margin_p": p3,
        "recommendation": interpret_result(p1, p2, p3)
    }

In [20]:
# Define hypotheses to test
hypotheses = [
    ("Province", "H₀: No risk difference across provinces"),
    ("PostalCode", "H₀: No risk difference between zip codes"),
    ("PostalCode", "H₀: No margin difference between zip codes"),
    ("Gender", "H₀: No risk difference between Women and Men"),
]

# Run hypothesis testing for each
for group_col, hypothesis_name in hypotheses:
    # Drop missing data in relevant columns
    filtered = data.dropna(subset=['TotalClaims', 'TotalPremium', group_col])
    results = hypothesis_test_pipeline(filtered, group_col, hypothesis_name)
    print(results["recommendation"])


--- Hypothesis: H₀: No risk difference across provinces ---


C:\Users\smith\AppData\Local\Temp\ipykernel_2700\2500647585.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  margin = data.groupby('Group').apply(lambda x: (x['TotalPremium'] - x['TotalClaims']).mean())


Chi-squared test (Claim Frequency): p = 0.0000
T-test (Claim Severity): p = 0.0306
T-test (Margin): p = 0.1638
→ Risk frequency difference is significant. → Claim severity difference is significant.

--- Hypothesis: H₀: No risk difference between zip codes ---


C:\Users\smith\AppData\Local\Temp\ipykernel_2700\2500647585.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  margin = data.groupby('Group').apply(lambda x: (x['TotalPremium'] - x['TotalClaims']).mean())


Chi-squared test (Claim Frequency): p = 0.0579
T-test (Claim Severity): p = 0.7002
T-test (Margin): p = 0.2445
Fail to reject null hypothesis. No significant differences found.

--- Hypothesis: H₀: No margin difference between zip codes ---


C:\Users\smith\AppData\Local\Temp\ipykernel_2700\2500647585.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  margin = data.groupby('Group').apply(lambda x: (x['TotalPremium'] - x['TotalClaims']).mean())


Chi-squared test (Claim Frequency): p = 0.0579
T-test (Claim Severity): p = 0.7002
T-test (Margin): p = 0.2445
Fail to reject null hypothesis. No significant differences found.

--- Hypothesis: H₀: No risk difference between Women and Men ---


C:\Users\smith\AppData\Local\Temp\ipykernel_2700\2500647585.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  margin = data.groupby('Group').apply(lambda x: (x['TotalPremium'] - x['TotalClaims']).mean())


Chi-squared test (Claim Frequency): p = 0.0189
T-test (Claim Severity): p = 0.0023
T-test (Margin): p = 0.2877
→ Risk frequency difference is significant. → Claim severity difference is significant.
